In [243]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm
from datetime import datetime

In [127]:
df=pd.read_csv("raw.csv")
df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())
CPCleanedUnique=list(df.CPCleaned.unique())

In [244]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

def exportStata(path=datetime.now().strftime("%Y%m%d%H%M")):
    df.to_stata(path, version=118)



# [convert](./convert.txt)

# grouping:
## machine grouping:

In [152]:
for i in CPCleanedUnique:
    print(df[df.CPCleaned==i].QCleaned.nunique(), "\t",i,sep="")

33	other grounds
32	worker protection
22	emergency preparedness
4	hazardous work and other worst forms
24	chemicals and hazardous substances
18	dialogue, discipline and disputes
28	paid leave
8	working environment
14	gender
13	minimum wages/piece rate wages
22	leave
37	social security and other benefits
5	collective bargaining
16	worker accommodation
8	method of payment
1	forced labour and overtime
8	interference and discrimination
11	coercion
25	termination
7	race and origin
11	religion and political opinion
2	freedom to associate
1	bonded labour
14	employment contracts
21	contracting procedures
16	regular hours
3	union operations
15	health services and first aid
17	welfare facilities
18	overtime
9	documentation and protection of young workers
4	premium pay
4	wage information, use and deduction
26	osh management systems
16	overtime wages
4	strikes
1	child labourers
4	prison labour


In [ ]:
for i in tqdm(CPCleanedUnique):
    if i not in ['union operations','freedom to associate','strikes','collective bargaining','interference and discrimination']:
        if df[df.CPCleaned==i].QCleaned.nunique()>1: # otherwise ml model stop due to error
            ruleList=groupSent(df[df.CPCleaned==i].QCleaned.unique())
            for j in ruleList:
                if sum([len(df[df.QCleaned==x]) for x in j])<=6476:
                    replace(j)
        # break

# manually grouping:

In [139]:
# undo("has the employer taken legally required measures to help workers with hiv/ aids-related illnesses?")

# undo("is sexual orientation a factor in decisions regarding pay?")

# undo("is an applicant's sexual orientation a factor in hiring decisions?")

# replace(['does the employer allow workers with aids-related illnesses to work for as long as medically fit in available, appropriate work?',
# 'does the employer allow workers with hiv/aids-related illnesses to work for as long as medically fit in available, appropriate work?']
# )

# replace(['has the employer taken all legally required measures to reasonably help workers with aids-related illnesses?',
# 'has the employer taken legally required measures to help workers with hiv/ aids-related illnesses?'])

# replace([
#     'is age a factor in hiring, employment, or termination?',
#     "is an applicant's age a factor in hiring decisions?"
# ])

# replace([
#     "is a worker's real or perceived hiv/aids status a factor in decisions regarding termination or retirement?",
# "is a worker's real or perceived hiv/aids status a factor in the employer's decisions regarding termination or retirement?"
# ])

# replace(["have all accommodations required by national law been made for physically disabled persons?",
# "has the employer made required accommodations for physically disabled workers?"])

# replace(["does the employer hire one disabled worker for every 100 workers?	",
#          "does the employer comply with legal requirements regarding the hiring of disabled workers?"],
#          "compliance on hiring disabled workers")

In [143]:
tab(0)

other grounds


Country,Bangladesh,Cambodia,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,
are a worker's family responsibilities a factor in the employer's decisions regarding termination or retirement of workers?,0,0,0,0,0,27,27
are disabled persons who apply for work evaluated according to their ability to perform the job?,0,2237,0,0,119,1802,4158
are family responsibilities a factor in decisions regarding opportunities for promotion or access to training?,0,0,0,0,0,27,27
are family responsibilities a factor in decisions regarding pay?,0,0,0,0,0,54,54
are hiv/aids tests required at hiring?,0,0,0,0,0,1802,1802
are workers harassed on the basis of their family responsibilities?,0,0,0,0,0,27,27
compliance on hiring disabled workers,0,3,851,474,0,0,1328
do recruitment materials such as job announcements or job application forms refer to age in a manner that could discourage older workers from applying?,0,0,0,0,0,721,721
"does the employer allow workers with aids-related illnesses to work for as long as medically fit in available, appropriate work?",0,0,0,0,2,1802,1804


In [190]:
#1:

# replace(['are electrical wires, switches and plugs properly installed, grounded, and maintained?',
#          'are electrical wires, cables, switches, plugs and equipment (e.g. transformer, generator, electrical panel, circuit breakers) properly installed, grounded (for equipment), and maintained?'])

# replace(['are workers effectively trained and encouraged to use the personal protective equipment that is provided?	',
#          'are workers effectively trained and encouraged to properly use personal protective equipment and machines?'])

# replace(
#     ['are workers effectively trained to use the personal protective equipment and machines?',
# 'are workers effectively trained and encouraged to properly use personal protective equipment, machines and equipment?',
# 'are workers effectively trained and encouraged to properly use the personal protective equipment and machines?',
# 'are workers effectively trained and encouraged to use the personal protective equipment that is provided?',
# 'are workers effectively trained on occupational health and safety?',
# 'are workers effectively trained to use the personal protective equipment and machines?',
# 'are workers effectively trained to use the personal protective equipment that is provided?',
# 'are all workers trained and obligated to use machines and equipment safely?']
# )

# undo("are boilers registered or certified, as legally required?")

# replace(["do workers have suitable chairs?",
# "does the employer comply with ergonomic requirements?"])

# replace(["are electrical wires, cables, switches, plugs and equipment (e.g. transformer, generator, electrical panel, circuit breakers, electrical boilers) properly installed, grounded (for equipment), maintained, and certified?",
# "are electrical wires, cables, switches, plugs, and equipment (e.g. transformer, generator, electrical panel, circuit breakers) properly installed, grounded (for equipment), and maintained?",
# "are electrical wires, switches and plugs properly installed, grounded, and maintained?",
# "are electrical boxes, electrical wires, switches and plugs properly installed, grounded, and maintained?",
# "are electrical wires, switches and plugs properly installed, grounded, maintained and certified?",
# "are electrical wires, switches, plugs and appliances properly installed, grounded, and maintained?",
# "are electrical wires, switches, plugs and appliances properly installed, grounded, regularly checked and maintained?"])

# replace(["do the operators / technicians for boiler, generator, compressor, and pressure vessels have legally required permit and training?",
# "do steam generating machine operators have valid licenses?",
# "are boilers registered or certified, as legally required?",
# "do the operators/ technicians/ officers responsible for machinery/ equipment/ installations/ lifting equipment have the required license?",
# "do the technicians for electrical installations and boiler operators have the legally required license/ certification?"],
# "boiler or steam operator certified")

# replace(["are workers punished if they remove themselves from work situations that they believe present an imminent and serious danger to life or health?",
# "does the employer force workers to continue working when they have refused to work due to clear imminent and serious danger to their life or health?"],"prevent workers away from dangerous workplaces")

# replace(["does the employer provide workers with all necessary personal protective clothing and equipment?",
# "does the employer provide workers with all necessary personal protective clothing and equipment?"])

# undo("does the employer comply with legal requirements related to work subject to strict\noccupational safety requirements?")

# replace(["does the employer comply with legal requirements related to work subject to strict\noccupational safety requirements?",
# "does the employer comply with legal requirements related to work subject to strict occupational safety requirements?"])

# replace([
#     "does the employer comply with legal requirements related to machines and equipment subject to strict occupational safety requirements?",
# "does the employer comply with legal requirements related to machines, equipment subject to strict occupational safety requirements?"
# ])

# replace(['does the employer provide  workers with all necessary personal protective clothing and equipment?',
#        'does the employer provide workers with all necessary personal protective clothing and equipment?'])

# replace(["does the employer provide workers with all necessary personal protective clothing and equipment?",
# "has the employer provided workers who work with chemical substances with proper clothing and personal protective equipment when necessary?"])

# replace([
#     "boiler or steam operator certified",
#     "do workers use the personal protective equipment that is provided?"
# ])

# for i in ["has the employer provided workers with dust masks when necessary?",
# "has the employer provided workers with ear plugs when necessary?",
# "has the employer provided workers with metal gloves when necessary?",
# "has the employer provided workers with necessary ppe to protect against wet processes when necessary?",
# "has the employer provided workers with other ppe when necessary?",
# "has the factory conducted an assessment of the potential hazards in the workplace?	"]:
#     assign(i)

# df.loc[df.Q=="do the technicians for electrical installations and boiler operators have the legally required license/ certification?","QCleaned"]=\
# "do the technicians for electrical installations and boiler operators have the legally required license/ certification?"


# for i in [
# "do workers use the machines and equipment in a safe manner?",
# "does the employer comply with legal requirements related to machines, equipment subject to strict occupational safety requirements?",
# "does the employer comply with legal requirements related to work subject to strict\noccupational safety requirements?",
# "has the factory conducted an assessment of the potential hazards in the workplace?",
# "is the workplace overcrowded?"
# ]:
#     assign(i)

In [191]:
# df[(df.QCleaned=="are electrical wires, switches and plugs properly installed, grounded, and maintained?") & (df.Country=="Indonesia")].Q.value_counts()
# df[(df.QCleaned=="boiler or steam operator certified") & (df.Country=="Bangladesh")].Q.value_counts()
tab(1)

worker protection


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are all workers trained and obliged to use the personal protective equipment that is provided?,0,0,0,834,0,0,0,834
are appropriate safety warnings posted in the workplace?,584,2503,161,833,474,119,1802,6476
"are electrical wires, switches and plugs properly installed, grounded, and maintained?",584,2503,161,838,474,122,1802,6484
are proper guards installed and maintained on all dangerous machines and equipment?,584,2503,161,833,474,119,1802,6476
are workers effectively trained on occupational health and safety?,584,2286,161,833,474,119,1802,6259
are workers protected against falls from heights?,357,2241,0,480,0,0,1802,4880
boiler or steam operator certified,357,2434,0,834,0,122,1802,5549
do the technicians for electrical installations and boiler operators have the legally required license/ certification?,357,0,0,0,0,0,0,357
do workers have suitable chairs?,584,2321,161,833,474,122,1802,6297


In [225]:
#2:

# replace(["does the workplace have adequate and accessible fire-fighting equipment?",
# "does the workplace have adequate fire-fighting equipment?"])

# replace(["does the workplace have adequate fire-fighting equipment and within easy reach of workers?",
# "does the workplace have adequate fire-fighting equipment that is within easy reach of workers?"])

# replace([
#     "does the workplace have adequate fire-fighting equipment and within easy reach of workers?",
# "does the workplace have adequate fire-fighting equipment?"
# ])

# replace([
#     "has the employer appointed and trained a fire management team?	",
#     "has the employer trained at least 80% of workers to use the fire-fighting equipment?"
# ])

# for i in [
#     "accessable unlocked emergency exits",
#     "are emergency exits and escape routes clearly marked?",
#           "at least 2 possible emergency exits","fire detection and alarm system","periodic emergency drills"]:
#     discardYear(i,"Nicaragua")

# for i in [
#     "accessable unlocked emergency exits",
#           "at least 2 possible emergency exits","fire detection and alarm system",
#           "does the workplace have adequate fire-fighting equipment?",
#           "has the employer appointed and trained a fire management team?",
#           "periodic emergency drills"]:
#     discardYear(i,"Indonesia")

replace([
    "has the employer appointed and trained a fire management team?",
    "has the employer trained an appropriate number of workers to use the fire-fighting equipment?"
])

In [226]:
tab(2)

emergency preparedness


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
accessable unlocked emergency exits,584,2503,161,833,474,119,1802,6476
are emergency exits and escape routes clearly marked?,584,2296,161,833,474,119,1802,6269
are escape routes free of obstruction?,0,2503,0,0,0,0,0,2503
are flammable materials safely stored?,584,2254,161,833,474,117,1802,6225
are floors and corridors in line with legal requirements?,0,0,0,0,0,117,0,117
are possible sources of ignition appropriately safeguarded?,584,2241,161,833,474,117,1802,6212
"are the emergency exits locked during working hours, including overtime?",0,2503,0,0,0,0,0,2503
at least 2 possible emergency exits,584,2503,161,833,474,119,1802,6476
does the employer comply with emergency requirements?,357,0,0,480,0,0,0,837


In [227]:
tab(3)

hazardous work and other worst forms


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are any workers under age 18 subjected to the unconditional worst forms of child labour?,584,2236,0,833,474,119,1802,6048
are any workers who are under age 18 working at night?,584,2236,161,833,474,119,1802,6209
are any workers who are under age 18 working overtime?,0,0,161,0,0,0,0,161
do workers under the age of 18 perform work that is hazardous by nature?,584,2236,161,833,0,119,1802,5735
All,1752,6708,483,2499,948,357,5406,18153


---


In [259]:
#4:
# these 2 discard code should not be executed:
# discardYear("are chemicals and hazardous substances properly labeled?","Nicaragua")
# discardYear("does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace?","Nicaragua")
# "has the employer effectively trained workers who work with chemicals and hazardous substances?"

# replace(
#     ["has the employer effectively trained workers and supervisor who work with or are responsible for hazardous chemicals?",
# "has the employer effectively trained workers and supervisors who work with or are responsible for hazardous chemicals?",
# "has the employer effectively trained workers who work with chemicals and hazardous substances?",
# "has the employer trained all workers who work with chemicals and hazardous substances?"]
# )

# replace(
# ["has the employer taken action to assess, monitor, prevent and limit workers' exposure to chemicals and hazardous substances?",
# "has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?	",
# "has the employer taken action to, prevent and limit workers' exposure to chemicals and hazardous substances?"]
# )

# replace(
#     [
#         "does the employer keep chemical safety records for the hazardous chemicals used in the workplace?",
# "does the employer have chemical safety data sheets for all the hazardous chemicals used in the workplace?",
# "does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace, and are the chemicals properly labelled?",
# "does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace?	"
#     ]
# )


# replace([
# "does the employer keep a chemical inventory, and ensure that all chemicals and hazardous substances are properly labelled and have safety data sheets?",
# "does the employer keep an inventory of chemicals and hazardous substances used in the workplace?",
# "does the employer keep an inventory of hazardous chemicals used in the workplace?"
# ])

# replace(
#     [
#         "does the employer provide adequate washing facilities and cleaning materials in the event of exposure to hazardous chemicals?",
#         "does the employer provide adequate washing facilities and cleansing materials in the event of exposure to hazardous chemicals?"
#     ]
# )

In [260]:
tab(4)

chemicals and hazardous substances


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are chemical mixing rooms properly equipped and used?,0,2238,0,0,0,0,0,2238
are chemicals and hazardous substances properly labeled?,584,2503,161,0,0,122,1802,5172
are chemicals and hazardous substances properly stored?,0,0,0,0,0,0,1802,1802
does the employer have the legally required license for storage and use of acids in the factory?,357,0,0,0,0,0,0,357
does the employer keep an inventory of hazardous chemicals used in the workplace?,584,2293,161,833,474,119,1802,6266
does the employer keep chemical safety records for the hazardous chemicals used in the workplace?,584,2279,161,851,0,122,1802,5799
does the employer provide adequate washing facilities and cleaning materials in the event of exposure to hazardous chemicals?,584,2276,161,833,474,119,1802,6249
does the employer provide information on the chemicals and hazardous substances used in the workplace to the ministry of labour?,0,0,0,0,0,119,0,119
has the employer appointed a chemical officer?,0,0,0,834,0,0,0,834


In [272]:
#5:
# replace(
#     ["did the employer resolve collective disputes in compliance with legal requirements?",
# "did the employer resolve grievances and disputes in compliance with legal requirements?"]
# )

# replace(
#     ["do disciplinary measures comply with legal requirements?",
# "do the disciplinary measures comply with the company's by-laws approved by the ministry of labor?",
# "are disciplinary measures proportional to the seriousness of workers' misconduct?"]
# )

# replace([
#     "if union labour committee is formed, does employer meet worker representatives on a regular basis?",
#     "does the factory have a functioning participation committee?"
# ])

# replace([
# "does the factory have a functioning bipartite cooperation institution?",
# "does the factory have a functioning participation committee?"
# ])

# replace(
#     ["did the employer resolve grievances and disputes in compliance with legal requirements?",
# "did the employer resolve individual grievances and disputes in compliance with legal requirements?"]
# )

# replace(["does the employer comply with requirements regarding shop stewards?",
#          "does the factory have a functioning participation committee?"], "have participation committee or shop steward")

# assign("did the employer resolve collective disputes in compliance with legal requirements?")

In [273]:
tab(5)

dialogue, discipline and disputes


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
bullying harassment,584,2503,161,851,474,122,1802,6497
did the employer resolve grievances and disputes in compliance with legal requirements?,584,2503,161,833,474,0,1802,6357
did the factory consult the measures (suspension or temporary closed) to be taken with the workers' representatives? \ndid they reach an agreement?\ndid you present the agreement to the ministry of labor?,0,0,0,0,0,3,0,3
do disciplinary measures comply with legal requirements?,584,2236,0,833,474,119,1802,6048
does dialogue at the workplace take place in compliance with legal requirement?,0,0,0,0,0,0,1802,1802
does the factory have a qualified welfare officer?,584,0,0,0,0,0,0,584
has the employer complied with arbitration awards?,357,2503,0,0,0,0,0,2860
has the employer complied with court orders arising from individual or collective disputes?,0,2237,0,0,0,0,0,2237
has the employer complied with final conciliation agreements?,0,2503,0,0,0,0,0,2503


In [290]:
#6:
# undo("does the employer pay workers correctly during daily breaks?")

# replace([
#     "does the employer pay workers correctly during the daily break?",
#     "does the employer pay workers correctly during daily breaks?"
#     ])

# replace(
#     [
# "does the employer pay workers correctly for annual leave (14 or 21 days)?",
# "does the employer pay workers correctly for annual leave (vacation)?",
# "does the employer pay workers correctly for annual leave?",
# "does the employer pay workers correctly for legally required annual leave?",
# "does the employer pay workers correctly during annual leave?"
#     ], "legal pay for annual leave"
# )

# replace([
#     "paid leave for breastfeed",
#     "does the employer pay workers for one hour breastfeeding break per day?"
# ])

# replace([
#     "does the employer provide paid sick leave as required by the factory’s internal regulations, or if there are no internal regulations, according to the ministry's practice?",
# "paid sick leave","does the employer settle claims for sick leave and maternity leave within 3 working days?",
# "does the employer pay workers correctly during medical leave?"
# ]
# )

# replace([
#     "does the employer pay workers correctly during suspensions of work?",
#     "does the employer pay workers correctly during work stoppages?	"
# ],"pay for work stoppage")

replace([
    "does the employer pay workers correctly during personal leave?",
    "does the employer pay workers correctly for personal leave?"
])

In [295]:
tab(6)

paid leave


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
do pregnant workers receive correct payment when they take time off for prenatal care?,0,0,0,0,0,0,1802,1802
do workers receive correct payment during sick leave and other types of leave that are covered by social insurance?,0,0,0,0,0,0,1802,1802
does the employer pay any workers incorrectly for any types of paid time off (breaks and leave)?,0,0,60,0,0,0,0,60
does the employer pay for paternity leave when required?,0,0,0,0,0,117,1802,1919
does the employer pay women workers for 30 minutes rest per day during their periods?,0,0,0,0,0,0,1802,1802
does the employer pay workers correctly during casual leave?,584,0,0,0,0,0,0,584
does the employer pay workers correctly during daily breaks?,0,0,81,0,0,119,0,200
does the employer pay workers correctly during special leave?,0,2503,0,0,0,0,0,2503
does the employer pay workers correctly for non-working weekly rest days?,0,0,0,0,0,119,0,119


In [298]:
df[(df.QCleaned=="paid sick leave") & (df.Country=="Vietnam")].Q.value_counts()

does the employer submit claims for sick leave and maternity leave to the social insurance agency within 10 days?    1555
does the employer settle claims for sick leave and maternity leave within 3 working days?                             247
Name: Q, dtype: int64

In [310]:
#7:
# replace([
#     "are noise levels acceptable?",
# "are noise levels measured by the employer and in line with applicable standard?	"
# ])

# replace([
#     "is the temperature and ventilation aceptable?",
# "is the temperature in the workplace acceptable?","is the workplace adequately ventilated?",
# "is the workplace climate measured by the employer, and in line with applicable standards?"
# ],"temperature and ventilation acceptable")


In [311]:
tab(7)

working environment


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are noise levels acceptable?,584,2371,161,833,474,122,1802,6347
is the workplace adequately lit?,584,2443,161,833,474,122,1802,6419
is the workplace adequately ventilated?,0,0,0,0,0,0,1802,1802
is the workplace clean and tidy?,584,2271,156,480,474,117,1802,5884
temperature and ventilation acceptable,584,2406,161,834,474,122,1802,6383
All,2336,9491,639,2980,1896,483,9010,26835


In [317]:
#8:

# replace([
#     "gender a factor in termination",
#     "is gender or marital status a factor in decisions regarding opportunities for promotion or access to training?"
# ])

# replace([
#     "does the employer terminate workers who are on maternity leave or force them to resign?",
#     "is gender or marital status a factor in the employer's decisions regarding termination or retirement of workers?"
# ],"maternity a factor in termination")

In [318]:
tab(8)

gender


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"does the employer change the employment status, position, wages, benefits or seniority of workers during maternity leave?",584,2236,161,833,0,122,1802,5738
gender a factor in promotion,584,2503,58,833,0,119,1802,5899
gender a factor in termination,584,2503,161,851,0,122,1081,5302
gender a factor regarding pay,584,2503,161,833,474,119,1802,6476
gender discrimination on conditions of work,584,2503,161,851,474,122,1802,6497
is an applicant's gender a factor in hiring decisions?,584,2503,161,851,0,122,1802,6023
job announcements refer to gender,584,2503,161,0,474,119,1802,5643
maternity a factor in termination,0,2503,161,0,0,0,0,2664
maternity leave included in continous service,584,2236,161,833,474,119,1802,6209


---
[haiti issue](./min-wages.png)

In [338]:
#9:
# replace(["does the employer pay piece rate workers correctly for ordinary hours of work when their piece rate earnings exceed minimum wage?",
# "does the employer pay piece rate workers correctly for ordinary hours of work?"])

# replace([
#     "does the employer pay piece rate workers correctly for ordinary hours of work?",
# "does the employer pay piece rate workers correctly for regular hours of work?",
# "are piece-rate workers paid correctly for ordinary hours of work?"
# ])

# undo("does the employer pay at least 90 gourdes per day for ordinary hours of work to apprentices?")


# replace(["does the employer pay at least 200 gourdes per day for ordinary hours of work to apprentices?",
# "does the employer pay at least 140 gourdes per day for ordinary hours of work to apprentices?",
# "does the employer pay at least 120 gourdes per day for ordinary hours of work to apprentices?",
# "does the employer pay at least 96 gourdes per day for ordinary hours of work to apprentices?",
# "does the employer pay at least 96 gourdes per day for ordinary hours of work to apprentices (120 gourdes as of 01 may 2016)?",  
# "does the employer pay at least 90 gourdes per day for ordinary hours of work to apprentices?",      
# "does the employer pay at least 90 gourdes (96 gourdes as of may 1st 2015) per day for ordinary hours of work to apprentices?"],
# "minimum pay to apprentices")

# replace(["does the employer pay at least 300 gourdes per day for ordinary hours of work to temporary workers?",
# "does the employer pay at least 240 gourdes per day for ordinary hours of work to temporary workers?",
# "does the employer pay at least 350 gourdes per day for ordinary hours of work to temporary workers?",
# "does the employer pay at least 500 gourdes per day for ordinary hours of work to temporary workers?",                                 
# "does the employer pay at least 240 gourdes per day for ordinary hours of work to temporary workers (300 gourdes as of 01 may 2016)?",   
# "does the employer pay at least 225 gourdes (240 gourdes as of may 1st, 2015) per day for ordinary hours of work to temporary workers?", 
# "does the employer pay at least 200 gourdes (225 gourdes as of may 1st, 2014) per day for ordinary hours of work to temporary workers?"],
# "minimum pay to temporary workers")

# replace(
#     ["does the employer pay at least minimum wage for all types of workers other than regular workers?",
#     "does the employer pay at least minimum wage for temporary and probationary workers?",
#     "does the employer pay higher than district minimum wage for ordinary hours of work to permanent (pkwtt) and non-permanent workers (pkwt) who have worked for more than 1 year?",
#     "minimum pay to temporary workers"]
# )

In [339]:
tab(9)

minimum wages/piece rate wages


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are piece-rate workers paid correctly for ordinary hours of work?,584,2503,126,833,474,119,1802,6441
"does the employer establish a wage structure and wage scale, announce it to all workers, and submit it to the local manpower office?",0,0,0,480,0,0,0,480
does the employer pay apprentices the wage agreed to by the parties?,584,0,0,0,0,0,1802,2386
does the employer pay at least 7% higher than the applicable legal minimum wage for ordinary hours of work to workers who have received vocational training?,0,0,0,0,0,0,1802,1802
is the piece rate set at a level that permits workers of average ability working normal hours to earn minimum wage?,0,2284,0,0,0,0,0,2284
minimum pay to apprentices,0,0,161,0,0,0,0,161
minimum pay to temporary workers,584,2503,161,833,474,121,1802,6478
minimum wages for regular workers,584,2503,161,851,474,122,1802,6497
All,2336,9793,609,2997,1422,362,9010,26529


In [371]:
#10:
# df.loc[df.Q=="does the employer provide required leave for personal reasons?", "QCleaned"]="does the employer provide required leave for personal reasons?"

# replace(
#     ["does the employer provide required annual leave to workers with at least 12 months' experience?",
# "does the employer provide required annual leave?	"]
# )

# replace([
#     "does the employer provide workers at least 14 days of paid annual leave per year, or 21 days after 5 years of service?",
# "does the employer provide workers at least 14 working days of annual leave per year, or 21 working days after 5 years of service?"
# ])

# replace([
#     "does the employer provide required maternity leave?",
#     "does the employer provide the correct amount of time off for maternity rest, including in cases of miscarriage?"
# ])

# replace([
#     "does the employer provide required sick leave?	",
#     "does the employer comply with the entitlement to sick leave and other types of leave that are covered by social insurance?	"
# ])

# replace([
#     "does the employer give workers at least 18 days of annual leave per year and an extra day of annual leave for every three years of service?",
#     "does the employer provide 12 days of annual leave per year?",
#     "does the employer provide required annual leave?",
#     "does the employer provide workers at least 14 days of paid annual leave per year, or 21 days after 5 years of service?",
#     "does the employer provide 15 days of annual leave to workers for every six months of continuous service in the factory?"
# ])

# df.loc[(df.QCleaned=="does the employer provide required annual leave?") & (df.Country=="Nicaragua"),"QCleaned"]="does the employer provide other types of required leave?"

# replace([
#     "does the employer provide required annual leave?",
#     "does the employer provide 15 days of annual leave to workers for every six months of continuous service in the factory?"
# ])

# replace([
#     "can female workers take time off when they feel pain during the first two days of menstruation and they notify the employer?",
# "does the employer allow workers to take 30 minutes rest during their period?"
# ],"period break")

# df.loc[df.Q=="does the employer provide required personal leave?","QCleaned"]="does the employer provide required personal leave?"

# replace(["does the employer provide required leave for personal reasons?",
#          "does the employer provide required personal leave?"])

In [379]:
tab(10)

leave


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are pregnant workers provided time off for prenatal visits?,0,0,0,0,0,0,1802,1802
does the employer give workers up to 7 days of time off for special leave per year?,0,2503,0,0,0,0,0,2503
does the employer provide 1 hour off to elderly workers in their last year of work before retirement?,0,0,0,0,0,0,1775,1775
does the employer provide 10 days of casual leave per year?,584,0,0,0,0,0,0,584
does the employer provide at least 15 days of annual leave to workers with 1 year of service?,0,0,161,0,0,0,0,161
does the employer provide other types of required leave?,0,0,0,0,0,2,0,2
does the employer provide required annual leave?,584,2503,161,851,474,122,1802,6497
does the employer provide required breastfeeding breaks?,0,2294,101,833,474,119,1802,5623
does the employer provide required maternity leave?,584,2503,161,851,474,122,1802,6497


In [421]:
#11:
# replace(["does the employer provide round trip transport for female workers who work between 11 p.m. and 5 a.m?",
# "does the employer provide transport home or a place to sleep for workers who finish work between 22:00 and 05:00?"])

# replace([
#     "are the welfare fund and workers’ welfare foundation fund used in the manner determined by the fund board of trustees?",
#     "are the workers' participation and welfare funds used and distributed as required?"
# ])

# df.loc[df.Q=="does the employer pay the required employer contribution to ona?","QCleaned"]="does the employer pay the required employer contribution to ona?"

# df.loc[df.Q=="does the employer pay 3% of workers' basic salary to ofatma for maternity and health insurance?","QCleaned"]="pay for maternity insurance"

# undo(
#     "does the employer correctly calculate and send the employer contribution to the social security corporation?"
# )

# undo("does the employer pay the required contributions for social insurance (bpjs ketenagakerjaan) and health care funds (bpjs kesehatan) to bpjs?")

# replace([
#     "Does the employer pay the required contributions for social insurance (BPJS Ketenagakerjaan) and health care funds (BPJS Kesehatan) to BPJS?".lower(),
#     "Does the employer comply with all requirements related to National healthcare funds (BPJS kesehatan), including registering workers, paying employer contributions, and collecting and forwarding workers’ contribution?".lower()
# ],"pay social insurance")

# replace([
#     "Does the employer collect and forward all workers' contributions for social insurance (BPJS Ketenagakerjaan) and health care funds (BPJS Kesehatan)?".lower(),
#     "Does the employer comply with all requirements related to national social insurance (BPJS Ketenagakerjaan), including registering workers, paying employer contributions, and collecting and forwarding workers' contributions?".lower()
# ],"forward social insurance")

# replace([
#     "Does the employer pay all workers who work regularly the correct attendance bonus during leave legally required by law?".lower(),
#     "Does the employer pay all workers who work regularly the correct attendance bonus when workers take annual leave?".lower()
# ])

# df.loc[df.Q=="if a provident fund has been established, does the employer match the contributions made by workers?","QCleaned"]="if a provident fund has been established, does the employer match the contributions made by workers?"
# replace(['has the employer established a provident fund upon written demand by workers?'],"collect and deposit workers' contributions into the fund")
# df.loc[df.Q=="has the employer established a provident fund upon written demand by workers?","QCleaned"]="has the employer established a provident fund upon written demand by workers?"

# replace([
#     "does the employer pay workers two festival bonuses per year?",
#     "does the employer pay workers the religious holiday allowance?"
# ])

In [420]:
collapse(
    "does the employer pay the required amount each year into the workers' participation fund?",
    "Bangladesh"
)

has the employer properly established a workers' participation fund and welfare fund, and paid the required amount into the funds each year?    566
does the employer pay the required amount each year into the welfare fund and workers’ welfare foundation fund?                                  18
does the employer pay the required amount each year into the workers' participation fund?                                                        18
Name: Q, dtype: int64

In [422]:
tab(11)

social security and other benefits


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are the workers' participation and welfare funds used and distributed as required?,584,0,0,0,0,0,0,584
collect and deposit workers' contributions into the fund,584,0,0,0,0,0,0,584
do workers receive required distributions from the workers' participation fund?,18,0,0,0,0,0,0,18
does the employer collect and forward workers' contributions to social insurance funds on time?,0,0,0,0,0,120,1802,1922
does the employer comply with other wage payments?,18,0,0,0,0,0,0,18
does the employer comply with requirements when workers have occupational accidents and diseases?,0,0,0,0,0,0,1802,1802
does the employer correctly calculate and send the employer contribution to the social security corporation?,0,0,0,0,474,0,0,474
"does the employer correctly calculate, collect, and forward workers' contributions to the social security corporation?",0,0,0,0,474,0,0,474
does the employer forward workers contributions to ofatma?,0,0,161,0,0,0,0,161


In [423]:
df.to_csv("r1.csv", index=False)